In [2]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric

ee.Initialize()

In [33]:
def func1(img):
    img = ee.Image(img)
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_z = img.getInfo()['properties']['MEAN_SOLAR_ZENITH_ANGLE']
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000 # i.e. Py6S uses units of kilometers
    # Instantiate
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = solar_z # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    def spectralResponseFunction(bandname):        
        
        bandSelect = {
            'B1':PredefinedWavelengths.S2A_MSI_01,
            'B2':PredefinedWavelengths.S2A_MSI_02,
            'B3':PredefinedWavelengths.S2A_MSI_03,
            'B4':PredefinedWavelengths.S2A_MSI_04,
            'B5':PredefinedWavelengths.S2A_MSI_05,
            'B6':PredefinedWavelengths.S2A_MSI_06,
            'B7':PredefinedWavelengths.S2A_MSI_07,
            'B8':PredefinedWavelengths.S2A_MSI_08,
            'B8A':PredefinedWavelengths.S2A_MSI_09,
            'B9':PredefinedWavelengths.S2A_MSI_10,
            'B10':PredefinedWavelengths.S2A_MSI_11,
            'B11':PredefinedWavelengths.S2A_MSI_12,
            'B12':PredefinedWavelengths.S2A_MSI_13,
                        }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN = info['SOLAR_IRRADIANCE_'+bandname]
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    ca  = surface_reflectance('B1')
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    nir = surface_reflectance('B8')
    swir1 = surface_reflectance('B11')
    swir2 = surface_reflectance('B12')
    
    ref = img.select('QA60').addBands(ca).addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    dateString = scene_date.strftime("%Y-%m-%d")
    ref = ref.copyProperties(img).set({              
                  'AC_date':dateString,
                  'AC_aerosol_optical_thickness':aot,
                  'AC_water_vapour':h2o,
                  'AC_version':'py6S',
                  'AC_contact':'ndminhhus@gmail.com',
                  'AC_ozone':o3})
    

    # define YOUR assetID 
    # in my case it was something like this..
    #assetID = 'users/samsammurphy/shared/sentinel2/6S/ESRIN_'+dateString
    #assetID = 'users/ndminhhus/eLEAF/nt/s2_SIAC/'+fname,
    # # export
    fname = ee.String(img.get('system:index')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ref,
        description= 'S2_BOA_'+fname,
        assetId = 'users/ndminhhus/eLEAF/nt/S2_py6S/'+'S2_BOA'+fname,
        region = region,
        scale = 10,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')

In [34]:
startDate = ee.Date('2018-01-01')
endDate = ee.Date('2020-01-01')
geom = ee.Geometry.Point(108.91220182000018,11.700863529688942)
geom1 = ee.Geometry.Point(108.8619544253213,11.40533502657536)# Ninh Thuan region for lower part (T49PBN)
geom2 = ee.Geometry.Point(108.96124878094292,11.799830246236146)# Ninh Thuan region for upper part (T49PBP)

region = geom.buffer(55000).bounds().getInfo()['coordinates']

In [35]:
# The Sentinel 2 image collection
#S2_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(geom1).filterDate(startDate,endDate).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50))
S2_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(geom2).filterDate(startDate,endDate).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50))

In [36]:
print(S2_col.size().getInfo())

60


In [17]:
S2_list = S2_col.toList(S2_col.size().getInfo())
img1 = ee.Image(S2_list.get(9))

In [18]:
toa = img1
boa = ee.Image(func1(toa))

In [20]:
print(boa.getInfo()['properties']['AC_date'])

2018-03-09


In [22]:
from IPython.display import display, Image

channels = ['B4','B3','B2']

original = Image(url=toa.select(channels).getThumbUrl({
                'region':region,
                'min':0,
                'max':3500
                }))

corrected = Image(url=ee.Image(boa).select(channels).getThumbUrl({
                'region':region,
                'min':0,
                'max':3500
                }))

display(original, corrected)

In [ ]:
col_length = S2_col.size().getInfo()
#print(col_length)

for i in range(0,col_length):
    #print(i)
    list = S2_col.toList(col_length)
    img = ee.Image(list.get(i))
    func1(img)


exporting 20180108T031109_20180108T032345_T49PBN--->done
exporting 20180123T032321_20180123T032316_T49PBN--->done
exporting 20180202T030931_20180202T031511_T49PBN--->done
exporting 20180207T030859_20180207T031350_T49PBN--->done
exporting 20180212T030831_20180212T032401_T49PBN--->done
exporting 20180217T030759_20180217T031916_T49PBN--->done
exporting 20180222T030731_20180222T031550_T49PBN--->done
exporting 20180227T030649_20180227T031252_T49PBN--->done
exporting 20180304T030621_20180304T031229_T49PBN--->done
exporting 20180309T030539_20180309T032200_T49PBN--->done
exporting 20180319T030539_20180319T031100_T49PBN--->done
exporting 20180329T030539_20180329T032339_T49PBN--->done
exporting 20180403T030541_20180403T031957_T49PBN--->done
exporting 20180413T030541_20180413T031057_T49PBN--->done
exporting 20180423T030541_20180423T031503_T49PBN--->done
exporting 20180428T030539_20180428T032437_T49PBN--->done
exporting 20180508T030539_20180508T031153_T49PBN--->done
exporting 20180518T030539_20180